## 一步步构建SQL流程

In [17]:
import akshare as ak
import pandas as pd
import sqlite3
import os

In [94]:
df_raw = ak.stock_financial_abstract(symbol="600519")

'''将指标作为df index'''
df_raw.index = df_raw['指标'].values 
df_raw = df_raw.iloc[:,1:]
df_raw.drop_duplicates(inplace = True)
df_raw = df_raw.iloc[:,1:]

'''表格转置'''
df_transposed = df_raw.T  # 现在：index=报告期, columns=指标
df_transposed.index.name = "report_date"
df_long = df_transposed.reset_index()

needed_metrics = ["营业总收入", "归母净利润", "扣非净利润"]
df_final = df_long[["report_date"] + needed_metrics].copy()

# 后续处理（转单位、加字段等）
for metric in needed_metrics:
    df_final[metric] = pd.to_numeric(df_final[metric], errors='coerce') / 1e8

df_final['stock_code'] = "600519"

rename_map = {
    "营业总收入": "revenue",
    "归母净利润": "net_profit",
    "扣非净利润": "non_recurring_net_profit"
}
df_final = df_final.rename(columns=rename_map)

# 选择最终列
final_cols = ['stock_code',  'report_date', 'revenue', 'net_profit', 'non_recurring_net_profit']
df_final = df_final[[col for col in final_cols if col in df_final.columns]]

print("\n✅ 清洗后数据:")
print(df_final.head())


# 重新连接数据库
conn = sqlite3.connect("../database/finance_agent.db")

# 存入（追加模式）
df_final.to_sql("financial_reports", conn, if_exists="append", index=False)

print(f"✅ 成功存入 {len(df_final)} 条记录！")

conn.close()

# 连接数据库
conn = sqlite3.connect("../database/finance_agent.db")

# 用 pandas 直接查（你不需要写 SQL！）
query = """
SELECT revenue 
FROM financial_reports 
WHERE stock_code = '600519' 
  AND report_date = '20240930'
"""

result_df = pd.read_sql(query, conn)

if not result_df.empty:
    revenue = result_df['revenue'].iloc[0]
    print(f"贵州茅台 2023 年营收是：{revenue:.2f} 亿元")
else:
    print("未找到数据")

conn.close()


✅ 清洗后数据:
  stock_code report_date      revenue  net_profit  non_recurring_net_profit
0     600519    20250930  1309.038896  646.267467                646.806164
1     600519    20250630   910.937626  454.029623                453.902476
2     600519    20250331   514.434506  268.474742                268.498837
3     600519    20241231  1741.440700  862.281464                862.409060
4     600519    20240930  1231.225426  608.275521                607.786415
✅ 成功存入 100 条记录！
贵州茅台 2023 年营收是：1231.23 亿元


In [85]:
DB_PATH = "../database/finance_agent.db"

# === 查询 2023年年报 ===
conn = sqlite3.connect(DB_PATH)
query = """
SELECT revenue 
FROM financial_reports 
WHERE stock_code = '600519' 
"""
result_df = pd.read_sql(query, conn)
conn.close()

result_df

,revenue


In [86]:
import sqlite3
import pandas as pd

DB_PATH = "../database/finance_agent.db"
conn = sqlite3.connect(DB_PATH)

# 查看数据库中有哪些表
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("数据库中的表：")
print(tables)

conn.close()

数据库中的表：
                name
0  financial_reports


In [92]:
import os
import sqlite3
import pandas as pd
import akshare as ak

os.makedirs("../database", exist_ok=True)
DB_PATH = os.path.abspath("../database/finance_agent.db")

'''获取数据'''
df_raw = ak.stock_financial_abstract(symbol="600519")

'''数据清洗'''
df_raw.index = df_raw['指标'].values 
df_raw = df_raw.iloc[:,1:]
df_raw.drop_duplicates(inplace=True)
df_raw = df_raw.iloc[:,1:]
df_transposed = df_raw.T
df_transposed.index.name = "report_date"
df_long = df_transposed.reset_index()

needed_metrics = ["营业总收入", "归母净利润", "扣非净利润"]
df_final = df_long[["report_date"] + needed_metrics].copy()

'数值转换'
for metric in needed_metrics:
    df_final[metric] = pd.to_numeric(df_final[metric], errors='coerce') / 1e8

df_final['stock_code'] = "600519"
rename_map = {
    "营业总收入": "revenue",
    "归母净利润": "net_profit",
    "扣非净利润": "non_recurring_net_profit"
}
df_final = df_final.rename(columns=rename_map)

# 选择列（加固：只保留存在的列）
final_cols = ['stock_code', 'report_date', 'revenue', 'net_profit', 'non_recurring_net_profit']
df_final = df_final[[col for col in final_cols if col in df_final.columns]]

print(f"✅ 清洗完成! 共 {len(df_final)} 条记录")
print("前2行数据:")
print(df_final.head(2))

# =============== 第 3 步：重建表 + 强制验证 ===============
print("\n💾 正在写入数据库...")

# 1. 删除旧表（如果存在）
conn = sqlite3.connect(DB_PATH)
conn.execute("DROP TABLE IF EXISTS financial_reports;")
conn.commit()  # 确保删除生效
conn.close()

# 2. 用当前DataFrame重建表
conn = sqlite3.connect(DB_PATH)
df_final.to_sql("financial_reports", conn, index=False)  # 自动创建新表
conn.commit()  # 关键：显式提交！

# 3. 立即验证（同一连接内）
verification = pd.read_sql("SELECT * FROM financial_reports LIMIT 3;", conn)
conn.close()

print(f"\n🔍 验证结果 (表中前3行):")
if verification.empty:
    print("❌ 验证失败! 表仍然为空")
    # 额外诊断：检查DataFrame是否真的有数据
    print("\n⚠️ 诊断: 检查DataFrame内容")
    print("DataFrame是否为空?", df_final.empty)
    print("DataFrame前3行:")
    print(df_final.head(3))
else:
    print(verification)
    print(f"\n🎉 成功! 数据已存入数据库")
    # 测试查询
    sample_date = verification['report_date'].iloc[0]
    print(f"\n🔍 测试查询 (日期={sample_date}):")
    conn = sqlite3.connect(DB_PATH)
    test_query = f"""
    SELECT revenue 
    FROM financial_reports 
    WHERE stock_code = '600519' 
      AND report_date = '{sample_date}'
    """
    result = pd.read_sql(test_query, conn)
    conn.close()
    print(result)

✅ 清洗完成! 共 100 条记录
前2行数据:
  stock_code report_date      revenue  net_profit  non_recurring_net_profit
0     600519    20250930  1309.038896  646.267467                646.806164
1     600519    20250630   910.937626  454.029623                453.902476

💾 正在写入数据库...

🔍 验证结果 (表中前3行):
  stock_code report_date      revenue  net_profit  non_recurring_net_profit
0     600519    20250930  1309.038896  646.267467                646.806164
1     600519    20250630   910.937626  454.029623                453.902476
2     600519    20250331   514.434506  268.474742                268.498837

🎉 成功! 数据已存入数据库

🔍 测试查询 (日期=20250930):
       revenue
0  1309.038896


In [93]:
sample_date = '20250331'
conn = sqlite3.connect(DB_PATH)
test_query = f"""
SELECT revenue 
FROM financial_reports 
WHERE stock_code = '600519' 
    AND report_date = '{sample_date}'
"""
result = pd.read_sql(test_query, conn)
conn.close()
print(result)

      revenue
0  514.434506


## qwen demo

In [1]:
# 安装（若未安装）

import akshare as ak
import pandas as pd
import sqlite3
import os
import json

In [2]:
# 统一数据库路径
DB_DIR = "../database"
os.makedirs(DB_DIR, exist_ok=True)
DB_PATH = os.path.abspath(f"{DB_DIR}/finance_agent.db")
print(f"📁 数据库路径: {DB_PATH}")

# 连接并重建表
conn = sqlite3.connect(DB_PATH)
conn.execute("DROP TABLE IF EXISTS financial_reports;")
conn.commit()
conn.close()
print("✅ 数据库已初始化")

📁 数据库路径: c:\Users\sunrui\Desktop\tk实习\database\finance_agent.db
✅ 数据库已初始化


In [3]:
def load_stock_data(stock_code: str):
    """从 AKShare 获取单只股票的财务数据并存入数据库"""
    print(f"🔄 加载 {stock_code} 数据...")
    try:
        df_raw = ak.stock_financial_abstract(symbol=stock_code)
        
        # 清洗（你的原逻辑）
        df_raw.index = df_raw['指标'].values 
        df_raw = df_raw.iloc[:,1:]
        df_raw.drop_duplicates(inplace=True)
        df_raw = df_raw.iloc[:,1:]
        df_transposed = df_raw.T
        df_transposed.index.name = "report_date"
        df_long = df_transposed.reset_index()
        
        needed_metrics = ["营业总收入", "归母净利润", "扣非净利润"]
        df_final = df_long[["report_date"] + needed_metrics].copy()
        
        for metric in needed_metrics:
            df_final[metric] = pd.to_numeric(df_final[metric], errors='coerce') / 1e8
        df_final['stock_code'] = stock_code
        
        rename_map = {
            "营业总收入": "revenue",
            "归母净利润": "net_profit",
            "扣非净利润": "non_recurring_net_profit"
        }
        df_final = df_final.rename(columns=rename_map)
        
        # 存入数据库
        conn = sqlite3.connect(DB_PATH)
        df_final.to_sql("financial_reports", conn, if_exists="append", index=False)
        conn.close()
        print(f"✅ {stock_code} 共存入 {len(df_final)} 条记录")
        return True
    except Exception as e:
        print(f"❌ {stock_code} 加载失败: {str(e)}")
        return False

# 加载示例股票（Week 1 覆盖足够场景）
stocks = {
    "贵州茅台": "600519",
    "五粮液": "000858",
    "宁德时代": "300750",
    "招商银行": "600036"
}

for name, code in stocks.items():
    load_stock_data(code)

🔄 加载 600519 数据...
✅ 600519 共存入 100 条记录
🔄 加载 000858 数据...
✅ 000858 共存入 108 条记录
🔄 加载 300750 数据...
✅ 300750 共存入 38 条记录
🔄 加载 600036 数据...
✅ 600036 共存入 99 条记录


In [4]:
def get_report_date(year: int, quarter: int = None) -> str:
    if quarter is None:
        return f"{year}1231"
    q_map = {1: "0331", 2: "0630", 3: "0930", 4: "1231"}
    return f"{year}{q_map[quarter]}"

def query_financial_metric(stock_code: str, metric: str, year: int, quarter: int = None):
    """查询财务指标（单位：亿元）"""
    report_date = get_report_date(year, quarter)
    conn = sqlite3.connect(DB_PATH)
    query = f"""
    SELECT {metric}
    FROM financial_reports
    WHERE stock_code = '{stock_code}'
      AND report_date = '{report_date}'
    """
    df = pd.read_sql(query, conn)
    conn.close()
    return df[metric].iloc[0] if not df.empty else None

# 测试
print("🔍 测试查询:")
print("茅台 2023 营收:", query_financial_metric("600519", "revenue", 2023))
print("宁德 2023 扣非:", query_financial_metric("300750", "non_recurring_net_profit", 2023))

🔍 测试查询:
茅台 2023 营收: 1505.6033031644997
宁德 2023 扣非: 400.916737


In [49]:
import requests
import json

class LLMClient:
    def __init__(self, model_name: str, api_key: str):
        self.model_name = model_name
        self.api_key = api_key
        # 支持的模型列表（按你实际使用的调整）
        self.is_qwen = "qwen" in model_name.lower()
        self.is_deepseek = "deepseek" in model_name.lower()
        
    def _call_qwen(self, messages, functions=None, **kwargs):
        url = "https://dashscope.aliyuncs.com/api/v1/services/aigc/text-generation/generation"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        
        # 构造 input
        input_data = {"messages": messages}
        if functions:
            input_data["functions"] = functions
            input_data["tool_choice"] = "required"  # 强制调用
        
        payload = {
            "model": self.model_name,
            "input": input_data,
            "parameters": {
                "result_format": "message"  # 必须是 "message"
            }
        }
        
        print("🔍 Qwen 请求 payload:")
        print(json.dumps(payload, indent=2, ensure_ascii=False))
        
        resp = requests.post(url, headers=headers, json=payload)
        
        if resp.status_code != 200:
            print("❌ API Error:", resp.text)
            raise RuntimeError(f"Qwen API error: {resp.status_code} - {resp.text}")
        
        result = resp.json()
        print("🔍 Qwen 响应:")
        print(json.dumps(result, indent=2, ensure_ascii=False))
        
        return result

    def _call_deepseek(self, messages, functions=None, **kwargs):
        # 注意：截至 2025 年，DeepSeek 官方 API **不支持 function calling**
        # 因此我们分两种模式处理：
        # 1. 如果有 functions → 强制用 Qwen（或报错）
        # 2. 如果无 functions → 可用 DeepSeek 生成自然语言
        
        if functions is not None:
            raise ValueError("DeepSeek API does not support function calling. Use Qwen for tool calls.")
        
        url = "https://api.deepseek.com/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        payload = {
            "model": self.model_name,
            "messages": messages,
            "temperature": kwargs.get("temperature", 0.3),
            "max_tokens": kwargs.get("max_tokens", 500)
        }
        resp = requests.post(url, headers=headers, json=payload)
        if resp.status_code != 200:
            raise RuntimeError(f"DeepSeek API error: {resp.text}")
        return resp.json()

    def chat(self, messages, functions=None, **kwargs):
        """
        统一调用接口
        - 如果需要 function calling，必须使用 Qwen 系列模型
        - 如果只需生成文本，可使用 Qwen 或 DeepSeek
        """
        if functions is not None:
            if not self.is_qwen:
                raise ValueError("Function calling requires a Qwen model (e.g., 'qwen-max').")
            result = self._call_qwen(messages, functions, **kwargs)
            # 解析 Qwen function_call
            msg = result['output']['choices'][0]['message']
            if 'function_call' in msg:
                return {
                    "type": "function_call",
                    "name": msg["function_call"]["name"],
                    "arguments": json.loads(msg["function_call"]["arguments"])
                }
            else:
                return {
                    "type": "text",
                    "content": msg["content"]
                }
        else:
            if self.is_qwen:
                result = self._call_qwen(messages, functions=None, **kwargs)
                content = result['output']['choices'][0]['message']['content']
            elif self.is_deepseek:
                result = self._call_deepseek(messages, **kwargs)
                content = result['choices'][0]['message']['content']
            else:
                raise ValueError(f"Unsupported model: {self.model_name}")
            
            return {"type": "text", "content": content}

In [50]:
# ====== 配置区（你只需改这里） ======
QWEN_API_KEY = 'sk-8b54a49bb19c45c69fc00f9b1b32d261'      # 替换为你的 Qwen API Key
DEEPSEEK_API_KEY = "sk-2815d0bd2377461f9c152d51d039b396"  # 替换为你的 DeepSeek API Key

# 选择模型（可随时切换）
FUNCTION_CALL_MODEL = "qwen-max"        # 必须用 Qwen 做 function calling
GENERATE_MODEL = "deepseek-chat"        # 可用 DeepSeek 生成自然语言

# 初始化客户端
func_client = LLMClient(FUNCTION_CALL_MODEL, QWEN_API_KEY)
gen_client = LLMClient(GENERATE_MODEL, DEEPSEEK_API_KEY)
# ===================================

In [51]:
FINANCIAL_FUNCTIONS = [
    {
        "name": "query_financial_metric",
        "description": "查询A股公司财务指标",
        "parameters": {
            "type": "object",
            "properties": {
                "stock_name": {
                    "type": "string",
                    "description": "公司名称，如'贵州茅台'"
                },
                "metric": {
                    "type": "string",
                    "enum": ["revenue", "net_profit", "non_recurring_net_profit"],
                    "description": "财务指标类型"
                },
                "year": {
                    "type": "integer",
                    "description": "年份，如2023"
                },
                "quarter": {
                    "type": "integer",
                    "enum": [1, 2, 3, 4],
                    "description": "季度（可选），1-4"
                }
            },
            "required": ["stock_name", "metric", "year"]  # 注意：必须放在 parameters 同级
        }
    }
]

STOCK_NAME_MAP = {
    "贵州茅台": "600519",
    "茅台": "600519",
    "五粮液": "000858",
    "宁德时代": "300750",
    "招商银行": "600036",
    "招行": "600036"
}

In [52]:
def handle_question(question: str):
    # Step 1: 用 Qwen 解析意图（function calling）
    messages = [{"role": "user", "content": question}]
    try:
        func_result = func_client.chat(messages, functions=FINANCIAL_FUNCTIONS)
        
        if func_result["type"] != "function_call":
            return func_result["content"]  # LLM 直接回答（如无法解析）
        
        # Step 2: 执行函数
        args = func_result["arguments"]
        stock_code = STOCK_NAME_MAP.get(args["stock_name"])
        if not stock_code:
            return f"未知公司: {args['stock_name']}"
        
        value = query_financial_metric(
            stock_code=stock_code,
            metric=args["metric"],
            year=args["year"],
            quarter=args.get("quarter")
        )
        
        # Step 3: 用 DeepSeek 生成自然语言（可换模型）
        context = (
            f"用户问题：{question}\n"
            f"结构化参数：{json.dumps(args, ensure_ascii=False)}\n"
            f"查询结果：{value:.2f} 亿元（若为 None 则表示数据缺失）\n"
            "请用中文自然语言回答，包含公司名、时间、指标和数值，并注明数据来源。"
        )
        gen_messages = [{"role": "user", "content": context}]
        gen_result = gen_client.chat(gen_messages)
        
        return gen_result["content"]
        
    except Exception as e:
        return f"处理出错: {str(e)}"

In [53]:
# 测试（确保填入了有效的 API Key）
print("💬 测试真实 API 调用:")
print(handle_question("贵州茅台2023年营收是多少？"))

💬 测试真实 API 调用:
🔍 Qwen 请求 payload:
{
  "model": "qwen-max",
  "input": {
    "messages": [
      {
        "role": "user",
        "content": "贵州茅台2023年营收是多少？"
      }
    ],
    "functions": [
      {
        "name": "query_financial_metric",
        "description": "查询A股公司财务指标",
        "parameters": {
          "type": "object",
          "properties": {
            "stock_name": {
              "type": "string",
              "description": "公司名称，如'贵州茅台'"
            },
            "metric": {
              "type": "string",
              "enum": [
                "revenue",
                "net_profit",
                "non_recurring_net_profit"
              ],
              "description": "财务指标类型"
            },
            "year": {
              "type": "integer",
              "description": "年份，如2023"
            },
            "quarter": {
              "type": "integer",
              "enum": [
                1,
                2,
                3,
                4
      

In [54]:
question = "贵州茅台2023年营收是多少？你要调用'query_financial_metric'工具来解决问题"
messages = [{"role": "user", "content": question}]
func_result = func_client.chat(messages, functions=FINANCIAL_FUNCTIONS)

if func_result["type"] != "function_call":
    print(func_result["content"]) # LLM 直接回答（如无法解析）

🔍 Qwen 请求 payload:
{
  "model": "qwen-max",
  "input": {
    "messages": [
      {
        "role": "user",
        "content": "贵州茅台2023年营收是多少？你要调用'query_financial_metric'工具来解决问题"
      }
    ],
    "functions": [
      {
        "name": "query_financial_metric",
        "description": "查询A股公司财务指标",
        "parameters": {
          "type": "object",
          "properties": {
            "stock_name": {
              "type": "string",
              "description": "公司名称，如'贵州茅台'"
            },
            "metric": {
              "type": "string",
              "enum": [
                "revenue",
                "net_profit",
                "non_recurring_net_profit"
              ],
              "description": "财务指标类型"
            },
            "year": {
              "type": "integer",
              "description": "年份，如2023"
            },
            "quarter": {
              "type": "integer",
              "enum": [
                1,
                2,
                3,
    

In [55]:
func_result

{'type': 'text',
 'content': '我将使用`query_financial_metric`工具来查询贵州茅台2023年的营收数据。不过，请注意，我的实际能力受限于我所拥有的信息以及能够访问的工具范围。在当前环境下，我并不能直接调用外部API或数据库查询最新的财务数据。但我可以模拟这一过程，并假设一个响应。\n\n```plaintext\n调用 query_financial_metric(公司="贵州茅台", 指标="营收", 年份=2023)\n```\n\n由于我无法真正执行这个函数调用来获取实时数据，请您自行查找最新发布的官方财报或通过可靠的财经新闻、证券分析网站获取最准确的信息。如果您需要帮助理解如何解读这些财务报告，我很乐意提供帮助！对于具体数值，建议参考贵州茅台官方发布的年度报告或相关公告。'}